In [11]:
import pandas as pd

# File paths
files = {
    "Half PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Half PPR Rankings.csv",
    "Full PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Full PPR Rankings.csv",
    "Non PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Non PPR Rankings.csv"
}

# Define weights
vbd_weight = 0.3
adp_weight = 0.7

# Loop through each file
for key, file in files.items():
    # Load the CSV file
    df = pd.read_csv(file)
    
    # Get the baseline points for each position
    baseline_qb = df[(df['Position'] == 'QB')]['Final PPG'].iloc[min(11, len(df[df['Position'] == 'QB'])-1)]
    baseline_rb = df[(df['Position'] == 'RB')]['Final PPG'].iloc[min(23, len(df[df['Position'] == 'RB'])-1)]
    baseline_wr = df[(df['Position'] == 'WR')]['Final PPG'].iloc[min(29, len(df[df['Position'] == 'WR'])-1)]
    baseline_te = df[(df['Position'] == 'TE')]['Final PPG'].iloc[min(11, len(df[df['Position'] == 'TE'])-1)]
    
    # Calculate VBD for each player based on their position
    def calculate_vbd(row):
        if row['Position'] == 'QB':
            return row['Final PPG'] - baseline_qb
        elif row['Position'] == 'RB':
            return row['Final PPG'] - baseline_rb
        elif row['Position'] == 'WR':
            return row['Final PPG'] - baseline_wr
        elif row['Position'] == 'TE':
            return row['Final PPG'] - baseline_te
        else:
            return 0
    
    df['VBD'] = df.apply(calculate_vbd, axis=1)
    
    # Normalize VBD and ESPN ADP
    df['Normalized VBD'] = (df['VBD'] - df['VBD'].min()) / (df['VBD'].max() - df['VBD'].min())
    df['Normalized ADP'] = (df['ESPN ADP'] - df['ESPN ADP'].min()) / (df['ESPN ADP'].max() - df['ESPN ADP'].min())
    
    # Calculate the weighted score
    df['Weighted Score'] = (vbd_weight * df['Normalized VBD']) + (adp_weight * (1 - df['Normalized ADP']))
    
    # Sort by weighted score in descending order to get the top players based on the weighted score
    df = df.sort_values(by='Weighted Score', ascending=False)
    
    # Adjust the ranking column
    df['Rank'] = range(1, len(df) + 1)
    
    # Display the top rows
    print(f"Top players in {key} with Weighted Score:")
    print(df.head(), "\n")
    
    # Save the updated DataFrame to a new CSV file
    output_file = file.replace(".csv", " with Weighted VBD.csv")
    df.to_csv(output_file, index=False)


Top players in Half PPR with Weighted Score:
    Unnamed: 0  Rank                 Name                 Team Position  \
31          31     1  Christian McCaffrey  San Francisco 49ers       RB   
39          39     2    Amon-Ra St. Brown        Detroit Lions       WR   
29          29     3          Breece Hall        New York Jets       RB   
18          18     4         Jahmyr Gibbs        Detroit Lions       RB   
32          32     5       Bijan Robinson      Atlanta Falcons       RB   

    Final PPG  Bye Week  ESPN ADP       VBD  Normalized VBD  Normalized ADP  \
31  14.415842         9       1.0  4.795390        0.919707        0.000000   
39  13.542963         5       7.0  5.932872        0.967013        0.024096   
29  14.744623        12       4.0  5.124171        0.933381        0.012048   
18  16.346483         5      14.0  6.726030        1.000000        0.052209   
32  14.032927        12       5.0  4.412475        0.903782        0.016064   

    Weighted Score  
31      